# PMT Waveform Analysis
**ICARUS** is a liquid argon detector currently in operation at Fermilab as part of the Short Baseline Neutrino (SBN) program. It consists of two symmetric modules, acting as large time-projection chambers (TPCs). Each of them houses 180 PMTs that collect scintillation light produced by charged particles crossing the liquid argon volume.

![](https://drive.google.com/uc?export=view&id=1-ONPfkpoUPfVOb9JjxTjaw1UOhgaHJad)

The photon detection system in ICARUS serves two primary purposes:
1.   **Triggering**: It allows to trigger on interesting events by setting a threshold on the amount of collected light.
2.   **Timing**: it provides accurate timing information on the event, i.e. the time of the neutrino interaction.

This notebook will guide you through a low-level analysis of PMT data taken at the ICARUS experiment. You will be looking at 180 PMT waveforms coming from a specific triggered event (run 9435, event 14386) in one of the ICARUS modules. You will start by looking at a few waveforms, explore a way to tag relevant light activity across all PMTs and extract a rough estimation of the position and time of the event.

You may be asked to fill in some Python code along the way, but don't be afraid to ask for help! ;)

Let's begin!

## Step 1: Setting up
Python offers a lot of ready-to-use libraries for scientific computing and data analysis. In the following cell, you are going to import some of them which you will need along the way.

In [ ]:
import numpy as np               # arrays, vectorize math functions
import matplotlib.pyplot as plt  # plotting
import pandas as pd              # dataframes for storing data

!pip install uproot3             # needed to read out .root files
import uproot3 as uproot         # uproot3 is deprecated, but I'm old school

The optical data is stored in a `.root` file. You can easily open files in Google Colab if they are stored in your Google Drive.

Do the following:
1.   Run the following cell and follow the instructions in the popup dialog to grant access to the Google Drive connected to your account.
2.   Go to [this Google Drive folder](https://drive.google.com/drive/folders/14SvWSV2c8bupNo5_yVhbXJ_kzsbUB6m_?usp=drive_link) and copy the data file to your Google Drive.
3.   Fill the path to your file in the code. You can also copy the path by esploring the navigation menu on the left side of this page.



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# add the path to the file in your Google Drive
# e.g.: "/content/drive/MyDrive/RENEW_DATA-Week_1/sample_data_icarus/opticalICARUS_run9435_ev14386.root"

file="/content/drive/MyDrive/RENEW_DATA-Week_1/sample_data_icarus/opticalICARUS_run9435_ev14386.root"

Now we can open the file and read what is inside.
We are going to store its contents in a `pandas` dataframe.

This is a very powerful tool for data science. You can think of it as an *smart* table (or better, a database) from which you can query what you need by selecting rows or columns satisfying certain conditions.

We will try to avoid using its full potential for the sake of code clarity. Apologies to the code purists out there!

In [ ]:
ttree = uproot.open(file) #opening with uproot
data = ttree["out_tree"].arrays(outputtype=pd.DataFrame, flatten=False) #convert into pd.DataFrame

# renaming some columns for better understanding
data.rename(columns={"wftstart":"t_start_us", "wfped_mean":"wf_baseline", "htstart":"h_tstart", "htmax":"h_tmax", "hamp":"h_amp", "hped":"h_ped"}, inplace=True)

Let's take a look at what we have by using the `head()` method, which returns the "head" of the table. By default, it shows the first 5 rows.

In [ ]:
print("Total number of columns:",len(data))
data.head()

Let's describe a little bit what we have:

*   Each row corresponds to one channel (`ch`). We have 180 PMTs, numbered from `0` to `180`.
*   `nsize` is the number of samples of the digitized PMT waveform. A sample is taken every `2 ns`, so each waveform is `12960*2 ns = 25960 ns = 25.960 us` long.
*   `t_start_us` is the start time of the first point of the waveform (in `us`).
*   `wf_baseline` is the mean baseline value of each waveform (in ADC counts). It's the value of the waveform in the absence of any signals.
*   `wf` is a vector containing the digitized values of the waveform (in ADC counts). The length of this vector is precisely `nsize`.

The other columns represent the output of a reconstruction algorithm on the waveform. The algorithm tries to look for "optical hits", i.e. significant light activity on the waveform.

* `nhits` is the number of optical hits found on the waveform.
* `h_tstart` is a vector of length `nhits` that contains the start time of each hit. This is not properly a time, but rather the sample number at which the hit starts.
* `h_tmax` is a vector of length `nhits` that contains the peak time of each hit. This is not properly a time, but rather the sample number at which the hit peaks.
* `h_amp` is a vector of length `nhits` that contains the amplitude (in ADC counts) of each hit.
* `h_ped` is a vector of length `nhits` that contains the pedestal of each hit. The baseline of the waveform right at the beginning of each hit.

## Step 2: Looking at raw waveforms
Numbers in a table are boring: let's get them out and plot a few waveforms!

You can choose an individual channel, extract the vectors from the dataframe and plot them with `matplotlib`. For a given channel, we need two vectors: time and ADC counts for each point of the sample waveform.

In [ ]:
# select a channel
channel = 116

Let's build a vector of times: we know the start time of the waveform, and then each sample is `2 ns` apart. The total lenght is given by `nsize`.

In [ ]:
# get waveform times
tstart = data[data.ch==channel].t_start_us.values # selects 't_start_us' value where 'ch' matches our selected channel
nsize = data[data.ch==channel].nsize.values # selects 'nsize' value where 'ch' matches our selected channel
tend = tstart + nsize*0.002 # time at the end of the waveform

t = np.arange(tstart, tend-0.002, 0.002) # build an array, starting at 'tstart' with 2 ns spacing
print(t)

Now we need the vector of values: this is already present in the dataframe, in the column `wf`.

In [ ]:
# get waveform values
y = data[data.ch==channel].wf.values[0] # selects 'wf' array where 'ch' matches our selected channel
print(y)

We are now ready to plot. Let's use `matplotlib`!

In [ ]:
# lets define a figure object with a nice size
fig = plt.figure(figsize=(14, 5))

# plot y vs t
plt.plot( t, y, label="PMT ch "+str(channel))

# always label your axis
plt.ylabel("Amplitude [ADC #]")
plt.xlabel("Time [us]")
plt.legend() # add a legend
plt.grid() # add a grid
plt.show() # show it

We finally see the waveform. As expected, the light pulses are negative and their amplitude can be hundreds of ADC counts.

It is also intersting to zoom around the main activity. To do so, you can use `plt.xlim((start,end))` to limit the range of the `x` axis.

### Exercise 1
Write a python function that plots a waveform taking as argument the PMT channel and the limits for the `x` axis. If no time limits are specified, show by default the entire waveform.

In [ ]:
def plot_waveform(data, channel, tmin=-1, tmax=-1):

    fig = plt.figure(figsize=(14, 5))

    # get waveform times
    # *** FILL HERE ***

    #get waveform data
    # *** FILL HERE ***

    # now plot
    # *** FILL HERE ***

    # let's also plot an horizontal line corresponding to the baseline level
    # we need to read it off from the dataframe
    baseline = # *** FILL HERE ***
    plt.axhline(y=baseline, color="red", linestyle="dotted")

    # here we also plot the beam "gate": the time in which we now
    # the beam impacted on the target and therefore neutrinos are expected
    trig_gate_diff = 1552 #ns
    beam_gate_width = 2200 #ns
    start_gate = 1500-(trig_gate_diff/1000.) #us
    end_gate = start_gate+beam_gate_width/1000. #us
    plt.axvline(x=start_gate,color="magenta",linestyle="dashed",lw=2,label="Start beam gate")
    plt.axvline(x=end_gate,color="orange",linestyle="dashed",lw=2,label="End beam gate")

    # deal with x-limits
    if tmin != -1 and tmax != -1:
        plt.xlim((tmin,tmax))

    plt.ylabel("Amplitude [ADC #]")
    plt.xlabel("Time [us]")
    plt.legend()
    plt.grid()
    plt.show()

Now test your function!

Try to look at different channels (e.g.: `4`, `53`, `66`, `122`, `158`). What do you see?



In [ ]:
channel=158
tmin = 1496
tmax = 1503

plot_waveform(data, channel, tmin, tmax)

## Step 3: Looking at optical hits
Raw waveforms are cool, but we need to extract information out of them to go further. The next step is then to identify significant peaks in the waveform and extract their time.

We call these negative peaks *optical hits*. As you have seen zooming on some waveforms, there can be complex structures. A scintillation event is in fact characterized by a prompt signal (which makes a big peak) followed by late photons distributed in time.

Let's find a way to select these negative peaks and save their time. First let's look at the following algorithm that looks for negative peaks:

In [ ]:
# inputs: our waveform (times, values, baseline) and a threshold
def find_negative_peaks(times, values, baseline, threshold):
    peak_times = []
    peak_values = []
    for i in range(1, len(values) - 1): # loops through each value

        # negative peak: lower than both its preceding and succeeding values
        # check if the value is above the given threshold (with respect to the baseline!)
        if values[i] < values[i - 1] and values[i] < values[i + 1] and (baseline-values[i]) > threshold:
            peak_times.append(times[i])
            peak_values.append(values[i])
    return peak_times, peak_values # results are saved in two arrays

You can test this simple function on a waveform:

In [ ]:
# get a waveform (you are experts now!)
channel = 116
tstart = data[data.ch==channel].t_start_us.values
tend = tstart + nsize*0.002
t = np.arange(tstart, tend-0.002, 0.002)
y = data[data.ch==channel].wf.values[0]
baseline = data[data.ch==channel].wf_baseline.values

In [ ]:
# choose a threshold
threshold = 40
peak_times, peak_values = find_negative_peaks(t,y,baseline,threshold)

print("Found "+str(len(peak_times))+" hits")
print(peak_times)
print(peak_values)

Plot them to see how they look:

In [ ]:
# lets define a figure object with a nice size
fig = plt.figure(figsize=(14, 5))

# plot y vs t
plt.plot( t, y, label="PMT ch "+str(channel))

#plot hits
plt.scatter( peak_times, peak_values, color="red", marker="o", label="Optical hits")

# always label your axis
plt.ylabel("Amplitude [ADC #]")
plt.xlabel("Time [us]")

tmin = 1499
tmax = 1503
plt.xlim((tmin,tmax))

plt.legend() # add a legend
plt.grid() # add a grid
plt.show() # show it

It seems to be working okay! However, the prompt peak is being splitted into may sub-hits, while it would make more sense to consider everything together as one hit.

Let's try comparing these hits with the ones already written in the file. Similarly as before, we need to read them off the dataframe.

In [ ]:
t_hits = tstart + data[data.ch==channel].h_tmax.values[0]*0.002 # waveform start time + sample * 0.002 us
v_hits = data[data.ch==channel].h_ped.values[0] - data[data.ch==channel].h_amp.values[0] # baseline - hit amplitude = hit value

print(t_hits)
print(v_hits)

Let's plot it!

In [ ]:
# lets define a figure object with a nice size
fig = plt.figure(figsize=(14, 5))

# plot y vs t
plt.plot( t, y, label="PMT ch "+str(channel))

#plot hits
plt.scatter( peak_times, peak_values, color="red", marker="o", label="Optical hits (simple function)")
plt.scatter( t_hits, v_hits, color="purple", marker=">", label="Optical hits (from file)")

# always label your axis
plt.ylabel("Amplitude [ADC #]")
plt.xlabel("Time [us]")

tmin = 1499
tmax = 1503
plt.xlim((tmin,tmax))

plt.legend() # add a legend
plt.grid() # add a grid
plt.show() # show it

### Exercise 2
Write a Python function that plots the reconstructed optical hits on a waveform taking as argument the PMT channel and the limits for the x axis. If no time limits are specified, show by default the entire waveform.

In [ ]:
def plot_optical_hits(data, channel, tmin, tmax):
  fig = plt.figure(figsize=(14, 5))

  # get waveform times
  # *** FILL HERE ***

  # get waveform data
  # *** FILL HERE ***

  # get hits (times & values)
  # *** FILL HERE ***

  # now plot
  # *** FILL HERE ***

  # deal with x-limits
  if tmin != -1 and tmax != -1:
    plt.xlim((tmin,tmax))

  plt.ylabel("Amplitude [ADC #]")
  plt.xlabel("Time [us]")
  plt.legend()
  plt.grid()
  plt.show()

Now test your function on different channels:

In [ ]:
channel=158
tmin = 1496
tmax = 1503

plot_optical_hits(data, channel, tmin, tmax)

## Step 4: Looking at optical flashes
Up until now you have been looking at one single PMT each time. However, neutrino events are expected to produce light activity in coincidence among many PMTs. Such collections of optical hits are called *flashes*.

Optical flashes are the true signature of interesting events. The goal is now to look to optical hits across all PMTs to find flashes. One way to do so visually is to make a scatter plot of every optical hits, using time in the x-axis and the PMT channel ID in the y-axis.

Let's try to make this plot!

In [ ]:
# there is a quick way to do this exploting pandas dataframes
# but let's fly low and make it simpler

# for each optical hit, we are going to add an entry to these arrays:
ch = [] # empty array for channel ID
tt = [] # empty array for ophit time

for channel in range(0,180): #loop through all channels

  #get opthit times
  tstart = data[data.ch==channel].t_start_us.values
  t_hits = tstart + data[data.ch==channel].h_tmax.values[0]*0.002
  # now t_hits is an array, but we want single hits
  for i in range(len(t_hits)):
    ch.append(channel)
    tt.append(t_hits[i])

In [ ]:
print(ch)
print(tt)

Now we can make the scatter plot:

In [ ]:
# lets define a figure object with a nice size
fig = plt.figure(figsize=(14, 5))

# plot channel_id vs time of each optical hit
plt.scatter(tt, ch, marker=".")

# lets add the beam time references
trig_gate_diff = 1552 #ns
beam_gate_width = 2200 #ns
start_gate = 1500-(trig_gate_diff/1000.) #us
end_gate = start_gate+beam_gate_width/1000. #us
plt.axvline(x=start_gate,color="magenta",linestyle="dashed",lw=2,label="Start beam gate")
plt.axvline(x=end_gate,color="orange",linestyle="dashed",lw=2,label="End beam gate")

# always label your axis
plt.title("OpHits in each channel vs time")
plt.xlabel("Time [us]")
plt.ylabel("PMT channel ID")

plt.legend() # add a legend
plt.grid() # add a grid
plt.show() # show it

Optical flashes are given by having multiple optical hits from different PMTs at the same time. In the plot you just made, they will therefore appear as **vertical lines** (different PMT IDs at the same time).

To better see how many flashes are present, you can set a threshold on the optical hits. This will be the goal of the exercise below.

### Exercise 3
Write a function to produce the scatter plot above selecting only the optical hits that are above a given threshold. Allow also the possibility to set limits for the x-axis. If no time limits are specified, show by default the entire range.

After that, test your function with different thresholds (note: ICARUS trigger threshold is 390 ADC counts):
1.   How many flashes do you see in data?
2.   Which flashes are likely to be neutrinos?

In [ ]:
def plot_optical_flashes(data, threshold, tmin, tmax):

  # empty arrays
  ch = []
  tt = []

  for channel in range(0,180): #loop through all channels

    # get ophit times
    # *** FILL HERE ***

    # get ophit amplitudes
    amp_hits = # *** FILL HERE ***

    for i in range(len(t_hits)):
      #check if amplitude above threshold
      if amp_hits[i] > threshold:
        ch.append(channel)
        tt.append(t_hits[i])

  # now let's plot
  fig = plt.figure(figsize=(14, 5))

  # plot channel_id vs time of each optical hit
  plt.scatter(tt, ch, marker=".")

  # deal with x-limits
  # *** FILL HERE ***

  # lets add the beam time references
  trig_gate_diff = 1552 #ns
  beam_gate_width = 2200 #ns
  start_gate = 1500-(trig_gate_diff/1000.) #us
  end_gate = start_gate+beam_gate_width/1000. #us
  plt.axvline(x=start_gate,color="magenta",linestyle="dashed",lw=2,label="Start beam gate")
  plt.axvline(x=end_gate,color="orange",linestyle="dashed",lw=2,label="End beam gate")

  # always label your axis
  plt.title("OpHits w/ threshold "+str(threshold)+" in each channel vs time")
  plt.xlabel("Time [us]")
  plt.ylabel("PMT channel ID")

  plt.legend() # add a legend
  plt.grid() # add a grid
  plt.show() # show it

Now test your function and play around with different thresholds:

In [ ]:
threshold = 100
tmin = 1495
tmax = 1510

plot_optical_flashes(data, threshold, tmin, tmax)

## Step 5: Event display
Great! We now have our flashes!

What happens next? Since ICARUS is a TPC, you would need to look at the images collected with the wires to see if you can spot interaction vertexes around the time of the flashes.

To match with the TPC, it is important to understand *where* the flash happened spatially inside the module. As you have seen from the previous plots, a flash is not a continuos vertical line: this is because PMTs see different regions of the detector depending on where they are placed.

Let's then try to draw the detector and highlight the PMTs related to one flash so we can see where it happened! In order to do so, we must first import a table that contains the `(x,y,z)` position for each PMT.

In [ ]:
# loading up geometry
geofile="/content/drive/MyDrive/RENEW_DATA-Week_1/sample_data_icarus/geofile.csv"
geo = pd.read_csv(geofile)

# display
geo.head()

Let's create three arrays storing PMT coordinates:

In [ ]:
xpmt = geo[geo.x<0].x.values
ypmt = geo[geo.x<0].y.values
zpmt = geo[geo.x<0].z.values

We can start setting up the 3D display. First, we draw the module and all PMTs in their actual positions.

In [ ]:
# prepare a 3D canvas
fig = plt.figure(figsize=(18, 10))
ax = fig.add_subplot(projection='3d')

# plot all PMTs
ax.scatter( zpmt, xpmt, ypmt, c='black', s=50, alpha=0.2  )

# make proportions better
ax.set_box_aspect(aspect = (10, 4, 2))

# label things
ax.set_ylabel("X (drift direction) [cm]", fontsize=12)
ax.set_xlabel("Z (beam direction) [cm]",fontsize=12)
ax.set_zlabel("Y (vertical direction) [cm]", fontsize=12)

fig.tight_layout()

We would like to highlight the PMTs involved in a flash. We can assign them a color and a size based on the amount of light they saw.

First, we need to select the PMTs and the hits involved in a flash. We can do so by looking at the previous plots and selecting only hits around the flash time.

In [ ]:
# go for the first flash
tmin = 1499.20 # approximately from zooming on the plots
tmax = 1499.45

ch_flash = []
amp_flash = []

for channel in range(0,180): #loop through all channels

  #get opthit times
  tstart = data[data.ch==channel].t_start_us.values
  t_hits = tstart + data[data.ch==channel].h_tmax.values[0]*0.002
  a_hits = data[data.ch==channel].h_amp.values[0]

  tt = [] # times of hits in flash
  aa = [] # amplitudes of hits in flash

  for i in range(len(t_hits)):

    if ( t_hits[i] < tmin or t_hits[i] > tmax): # skip if outside flash
      continue
    if ( a_hits[i] < 50): #skip if too low threshold
      continue

    aa.append(a_hits[i])
    tt.append(t_hits[i])

  if( len(tt) > 0): # if at least one hit passed the time selection
    ch_flash.append(channel) # this PMT saw the flash
    amp_flash.append(sum(aa)) # use sum of amplitude as (bad & rough) proxy for how much light it saw

In [ ]:
print(ch_flash)
print(amp_flash)

From the channel ID, we now need coordinates

In [ ]:
xpmt_flash = [ xpmt[i] for i in ch_flash ]
ypmt_flash = [ ypmt[i] for i in ch_flash ]
zpmt_flash = [ zpmt[i] for i in ch_flash ]

Now we can add them to the 3D plot!

In [ ]:
# prepare a 3D canvas
fig = plt.figure(figsize=(18, 10))
ax = fig.add_subplot(projection='3d')

# plot all PMTs
ax.scatter( zpmt, xpmt, ypmt, c='black', s=50, alpha=0.2  )

# plt PMTs in flash
ax.scatter ( zpmt_flash, xpmt_flash, ypmt_flash, s=np.array(amp_flash)*0.2, c=amp_flash, cmap='YlOrRd' )

# make proportions better
ax.set_box_aspect(aspect = (10, 4, 2))

# label things
ax.set_ylabel("X (drift direction) [cm]", fontsize=12)
ax.set_xlabel("Z (beam direction) [cm]",fontsize=12)
ax.set_zlabel("Y (vertical direction) [cm]", fontsize=12)

fig.tight_layout()

As you can see from the event display, the interaction related to this flash happened around `z = -250`, close to one of the walls as the PMTs in that region saw higher pulses. It is important to note that the amplitude of the pulse is not a good estimate for the total amount of light collected, as it does not consider the late photons in the tail. Similar displays are therefore made considering the integral of each pulse, which corresponds to the total charge collected by that PMT.

Try to repeat the same procedure for the other flashes you identified in Exercise 3. What do you see?